In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext('local')

22/07/26 10:27:55 WARN Utils: Your hostname, master resolves to a loopback address: 127.0.1.1; using 192.168.139.128 instead (on interface ens33)
22/07/26 10:27:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/26 10:27:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/26 10:27:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
rdd1 = sc.textFile('data/bb.txt')
rdd1.collect()

['10,20', '30,40', '50,60', '20,40']

In [39]:
rdd1.map(lambda v: v.split(',')).collect()

[['10', '20'], ['30', '40'], ['50', '60'], ['20', '40']]

In [5]:
rdd2 = rdd1.flatMap(lambda v: v.split(','))
rdd2.collect()

['10', '20', '30', '40', '50', '60', '20', '40']

In [6]:
rdd3 = rdd2.map(lambda v: int(v))
rdd3.collect()

[10, 20, 30, 40, 50, 60, 20, 40]

In [7]:
rdd4 = rdd3.distinct() #transform
rdd4.collect()

[10, 20, 30, 40, 50, 60]

In [40]:
myrdd = rdd1.flatMap(lambda v: v.split(',')).map(lambda v: int(v)).distinct()
myrdd.collect()

[10, 20, 30, 40, 50, 60]

In [41]:
myrdd.first() #action함수

10

In [42]:
# 비복원추출（３０％）
samRdd1 = myrdd.sample(False,0.3) # transform
samRdd1.collect()

[50, 60]

In [43]:
# 비복원　３개　추출
samList = myrdd.takeSample(False,3) #action
samList

[20, 50, 10]

### 통계 action 함수

In [44]:
myrdd.sum()

210

In [45]:
myrdd.mean()

35.0

In [46]:
myrdd.max()

60

In [47]:
myrdd.min()

10

In [49]:
myrdd.stdev() #표준편차

17.07825127659933

In [50]:
myrdd.stats()

(count: 6, mean: 35.0, stdev: 17.07825127659933, max: 60.0, min: 10.0)

In [54]:
# 10<=data<26.6, 26.6<=data<43.3, 43.3<=data<60
myrdd.histogram(3) #３개의 계급, 도수

([10.0, 26.666666666666668, 43.333333333333336, 60], [2, 2, 2])

In [56]:
# 0<=data<50, 50<=data<=100
myrdd.histogram([0, 50, 100])

([0, 50, 100], [4, 2])

In [59]:
def fn(a,b):
    print('a=',a)
    print('b=',b)
    print('='*20)
    return a+b

In [61]:
# 누적합
result = myrdd.reduce(fn)
result

a= 10
b= 20
a= 30
b= 30
a= 60
b= 40
a= 100
b= 50
a= 150
b= 60


210

In [ ]:
[10, 20, 30, 40, 50, 60]

In [63]:
# 누적합 lambda
result = myrdd.reduce(lambda a,b:a+b)
result

210

In [64]:
sc.stop()